# Controlling Agent Reasoning Loop with `return_direct` in Tools


In this tutorial, we'll explore how to control the reasoning loop of an agent using the `return_direct` option available in tools.

This feature plays a crucial role in streamlining the agent's decision-making process, particularly when immediate output from a single tool is sufficient for the task at hand rather than sending the output of tool to LLM for final response.

Setting `return_direct` to True impacts the agent's reasoning loop. When activated and the corresponding tool is called independently, the loop concludes, and the output from the tool is directly returned.

By the end of this tutorial, you'll have a comprehensive understanding of how to leverage the `return_direct` option to enhance the efficiency of an agent's reasoning loop.

[LlamaIndex YT Ref](https://www.youtube.com/watch?v=gFRbkRtLGZQ)

### Installation

In [1]:
# !pip install llama-index
# !pip install llama-index-llms-openai
# !pip install llama-index-llms-ollama

In [1]:
from typing import Optional

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core.bridge.pydantic import BaseModel

### Setup LLM

In [2]:
import os
llm = Ollama(model="Hermes2Pro", 
            request_timeout=30.0)

Here, we are defining a system to manage restaurant bookings using various functions and integrating them into a tool-based architecture.

The `Booking` class is a Pydantic model used to track and represent the state of a booking. It includes optional fields like name, email, phone, date, and time, allowing for flexible data entry.

`get_booking_state(user_id: str):` Retrieves the current state of a booking using the booking ID. It outputs the booking's details or a message if the ID is not found.

`update_booking(user_id: str, property: str, value: str):` Updates a specific property of a booking. It's designed to modify only the provided details.

`create_booking(user_id: str):` Initializes a new booking and stores it under a unique ID, prompting the user to provide further details.

`confirm_booking(user_id: str):` Finalizes the booking after ensuring all necessary information is provided. If any detail is missing, it raises an error.

In [3]:
# we will store booking under random IDs
bookings = {}

class Booking(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    phone: Optional[str] = None
    date: Optional[str] = None
    time: Optional[str] = None


def get_booking_state(user_id: str) -> str:
    """Get the current state of a booking for a given booking ID."""
    try:
        return str(bookings[user_id].dict())
    except:
        return f"Booking ID {user_id} not found"


def update_booking(user_id: str, property: str, value: str) -> str:
    """Update a property of a booking for a given booking ID. Only enter details that are explicitly provided."""
    booking = bookings[user_id]
    setattr(booking, property, value)
    return f"Booking ID {user_id} updated with {property} = {value}"


def create_booking(user_id: str) -> str:
    """Create a new booking and return the booking ID."""
    bookings[user_id] = Booking()
    return "Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time."


def confirm_booking(user_id: str) -> str:
    """Confirm a booking for a given booking ID."""
    booking = bookings[user_id]

    if booking.name is None:
        raise ValueError("Please provide your name.")

    if booking.email is None:
        raise ValueError("Please provide your email.")

    if booking.phone is None:
        raise ValueError("Please provide your phone number.")

    if booking.date is None:
        raise ValueError("Please provide the date of your booking.")

    if booking.time is None:
        raise ValueError("Please provide the time of your booking.")

    return f"Booking ID {user_id} confirmed!"

We will demonstrate two experiments:

**Experiment-1:** `return_direct` is not enabled in `get_booking_state_tool`

**Experiment-2:** `return_direct` is enabled in `get_booking_state_tool`

Once the entire booking process is complete and confirmed, we will review the booking details to understand how the final response differs when `return_direct` is enabled versus when it is disabled.

In both experiments, we will enable `return_direct` for `create_booking_tool` and `confirm_booking_tool` as we don't need the output to be sent to LLM for the final response.

### Experiment-1:

#### Create Tools

In [4]:
get_booking_state_tool = FunctionTool.from_defaults(fn=get_booking_state)
update_booking_tool = FunctionTool.from_defaults(fn=update_booking)
create_booking_tool = FunctionTool.from_defaults(fn=create_booking,
                                                 return_direct=True)
confirm_booking_tool = FunctionTool.from_defaults(fn=confirm_booking,
                                                  return_direct=True)

In [5]:
user = "user123"
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping {user} with making a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

#### Create Agent

In [6]:
worker = FunctionCallingAgentWorker(
    tools=[
        get_booking_state_tool,
        update_booking_tool,
        create_booking_tool,
        confirm_booking_tool,
    ],
    llm=llm,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

agent = AgentRunner(worker)

ValueError: Model name Hermes2Pro does not support function calling API. 

#### Let's create booking

In [ ]:
response = agent.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "user123"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [ ]:
response = agent.chat("Sure! My name is Ravi, and my email is ravi@gmail.com")

Added user message to memory: Sure! My name is Ravi, and my email is ravi@gmail.com
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "name", "value": "Ravi"}
=== Function Output ===
Booking ID user123 updated with name = Ravi
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "email", "value": "ravi@gmail.com"}
=== Function Output ===
Booking ID user123 updated with email = ravi@gmail.com
=== LLM Response ===
Thank you, Ravi. Could you please provide your phone number, and the date and time you would like to make the booking for?


In [ ]:
response = agent.chat("Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively.")

Added user message to memory: Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively.
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "phone", "value": "39429384923"}
=== Function Output ===
Booking ID user123 updated with phone = 39429384923
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "date", "value": "April 20th"}
=== Function Output ===
Booking ID user123 updated with date = April 20th
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "time", "value": "12PM"}
=== Function Output ===
Booking ID user123 updated with time = 12PM
=== Calling Function ===
Calling function: confirm_booking with args: {"user_id": "user123"}
=== Function Output ===
Booking ID user123 confirmed!


In [ ]:
response = agent.chat("provide booking details of user user123")

Added user message to memory: provide booking details of user user123
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "user123"}
=== Function Output ===
{'name': 'Ravi', 'email': 'ravi@gmail.com', 'phone': '39429384923', 'date': 'April 20th', 'time': '12PM'}
=== LLM Response ===
Here are the booking details for user123:

- Name: Ravi
- Email: ravi@gmail.com
- Phone: 39429384923
- Date: April 20th
- Time: 12PM


In [ ]:
print(response)

assistant: Here are the booking details for user123:

- Name: Ravi
- Email: ravi@gmail.com
- Phone: 39429384923
- Date: April 20th
- Time: 12PM


As you can see above the final booking details are sent to LLM for final response.

### Experiment-2:

#### Create Tools

In [ ]:
get_booking_state_tool = FunctionTool.from_defaults(fn=get_booking_state,
                                                 return_direct=True)
update_booking_tool = FunctionTool.from_defaults(fn=update_booking)
create_booking_tool = FunctionTool.from_defaults(fn=create_booking,
                                                 return_direct=True)
confirm_booking_tool = FunctionTool.from_defaults(fn=confirm_booking,
                                                  return_direct=True)

In [ ]:
user = "user123"
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping {user} with making a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

#### Create Agent

In [ ]:
worker = FunctionCallingAgentWorker(
    tools=[
        get_booking_state_tool,
        update_booking_tool,
        create_booking_tool,
        confirm_booking_tool,
    ],
    llm=llm,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

agent = AgentRunner(worker)

#### Let's create Booking

In [ ]:
response = agent.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "user123"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [ ]:
response = agent.chat("Sure! My name is Ravi, and my email is ravi@gmail.com")

Added user message to memory: Sure! My name is Ravi, and my email is ravi@gmail.com
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "name", "value": "Ravi"}
=== Function Output ===
Booking ID user123 updated with name = Ravi
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "email", "value": "ravi@gmail.com"}
=== Function Output ===
Booking ID user123 updated with email = ravi@gmail.com
=== LLM Response ===
Thank you, Ravi. Could you please provide your phone number, and the date and time you would like to make the booking for?


In [ ]:
response = agent.chat("Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively.")

Added user message to memory: Cool. Phone number is 39429384923, preferred data and time are April 20th and 12PM respectively.
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "phone", "value": "39429384923"}
=== Function Output ===
Booking ID user123 updated with phone = 39429384923
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "date", "value": "April 20th"}
=== Function Output ===
Booking ID user123 updated with date = April 20th
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "user123", "property": "time", "value": "12PM"}
=== Function Output ===
Booking ID user123 updated with time = 12PM
=== Calling Function ===
Calling function: confirm_booking with args: {"user_id": "user123"}
=== Function Output ===
Booking ID user123 confirmed!


In [ ]:
response = agent.chat("provide the booking details")

Added user message to memory: provide the booking details
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "user123"}
=== Function Output ===
{'name': 'Ravi', 'email': 'ravi@gmail.com', 'phone': '39429384923', 'date': 'April 20th', 'time': '12PM'}


In [ ]:
print(response)

{'name': 'Ravi', 'email': 'ravi@gmail.com', 'phone': '39429384923', 'date': 'April 20th', 'time': '12PM'}


As you can see the above response with `return_dict` enabled is directly from tool and not sent to LLM for final response